### 소리 증폭

In [ ]:
# import soundfile as sf
# import numpy as np
# import os

# # 파일 경로 설정 
# input_files = ['/content/정상.wav', '/content/비정상.wav', '/content/정상+비정상.wav']
# output_files = ['/content/정상_증폭.wav', '/content/비정상_증폭.wav', '/content/정상+비정상_증폭.wav']

# # 증폭 비율 설정 (2.0 = 2배 증폭)
# amplification_factor = 2.0

# # 파일 증폭 함수
# def amplify_audio(input_path, output_path, factor):
#     # 오디오 파일 읽기
#     data, samplerate = sf.read(input_path)
    
#     # 증폭 적용
#     amplified_data = data * factor
    
#     # 클리핑 방지 (값을 -1.0 ~ 1.0으로 제한)
#     amplified_data = np.clip(amplified_data, -1.0, 1.0)
    
#     # 증폭된 파일 저장
#     sf.write(output_path, amplified_data, samplerate)
#     print(f'{output_path} 파일이 증폭되어 저장되었습니다.')

# # 모든 파일 증폭
# for input_file, output_file in zip(input_files, output_files):
#     if os.path.exists(input_file):
#         amplify_audio(input_file, output_file, amplification_factor)
#     else:
#         print(f"{input_file} 파일이 존재하지 않습니다.")

### 소리 합성 (비정상 + 정상 , 동시에 발생)

In [2]:
pip install pydub

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


- 정상, 비정상 폴더만 있을 때 합치는 코드

In [ ]:
import os
import random
from glob import glob
from pydub import AudioSegment

# 디렉토리 설정
normal_dir = "종설데이터/정상"
abnormal_dir = "종설데이터/비정상"
output_dir = "종설데이터/혼합/비정상"
os.makedirs(output_dir, exist_ok=True)

# .wav 파일 전체 가져오기
normal_files = glob(os.path.join(normal_dir, "**", "*.wav"), recursive=True)
abnormal_files = glob(os.path.join(abnormal_dir, "**", "*.wav"), recursive=True)

# 생성할 혼합 데이터 수 (원하면 늘릴 수 있음)
mix_count = min(len(normal_files), len(abnormal_files), 30)

for i in range(mix_count):
    # 랜덤 선택
    normal_path = random.choice(normal_files)
    abnormal_path = random.choice(abnormal_files)

    # 파일 이름 추출 (확장자 제거)
    normal_name = os.path.splitext(os.path.basename(normal_path))[0]
    abnormal_name = os.path.splitext(os.path.basename(abnormal_path))[0]

    # 오디오 불러오기
    normal_audio = AudioSegment.from_wav(normal_path)
    abnormal_audio = AudioSegment.from_wav(abnormal_path)

    # 길이 맞추기 (짧은 쪽 기준)
    min_len = min(len(normal_audio), len(abnormal_audio))
    normal_audio = normal_audio[:min_len]
    abnormal_audio = abnormal_audio[:min_len]

    # 혼합
    mixed = normal_audio.overlay(abnormal_audio)

    # 저장
    out_name = f"혼합_동시_정상비정상_{abnormal_name}_{normal_name}_{i+1}.wav"
    out_path = os.path.join(output_dir, out_name)
    mixed.export(out_path, format="wav")
    print(f"Saved: {out_name}")

In [ ]:
# import os
# import random
# from glob import glob
# from pydub import AudioSegment

# # 디렉토리 설정
# normal_dir = "데이터/ㅈㅅ"
# abnormal_dir = "데이터/ㅂㅈㅅ"
# output_dir = "데이터/혼합"
# os.makedirs(output_dir, exist_ok=True)

# # .wav 파일 전체 가져오기
# normal_files = glob(os.path.join(normal_dir, "**", "*.wav"), recursive=True)
# abnormal_files = glob(os.path.join(abnormal_dir, "**", "*.wav"), recursive=True)

# # 생성할 혼합 데이터 수 (원하면 늘릴 수 있음)
# mix_count = min(len(normal_files), len(abnormal_files), 30)

# for i in range(mix_count):
#     # 랜덤 선택
#     normal_path = random.choice(normal_files)
#     abnormal_path = random.choice(abnormal_files)

#     # 파일 이름 추출 (확장자 제거)
#     normal_name = os.path.splitext(os.path.basename(normal_path))[0]
#     abnormal_name = os.path.splitext(os.path.basename(abnormal_path))[0]

#     # 오디오 불러오기
#     normal_audio = AudioSegment.from_wav(normal_path)
#     abnormal_audio = AudioSegment.from_wav(abnormal_path)

#     # 길이 맞추기 (짧은 쪽 기준)
#     min_len = min(len(normal_audio), len(abnormal_audio))
#     normal_audio = normal_audio[:min_len]
#     abnormal_audio = abnormal_audio[:min_len]

#     # 혼합
#     mixed = normal_audio.overlay(abnormal_audio)

#     # 저장
#     out_name = f"혼합_{abnormal_name}_{normal_name}_{i+1}.wav"
#     out_path = os.path.join(output_dir, out_name)
#     mixed.export(out_path, format="wav")
#     print(f"Saved: {out_name}")

Saved: 혼합_폭행4_일반3_1.wav
Saved: 혼합_폭행4_일반5_2.wav
Saved: 혼합_폭행5_일반5_3.wav
Saved: 혼합_폭행4_일반5_4.wav
Saved: 혼합_강도3_일반2_5.wav


c:\Users\seohyun\anaconda3\Lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


- 테스트용 (하나만)

In [ ]:
# ## 테스트용

# import os
# from pydub import AudioSegment

# # 파일 경로 설정 (원하는 파일 하나씩 직접 지정)
# normal_path = "ㅈㅅ/1-7456-A-13.wav"
# abnormal_path = "ㅂㅈㅅ/ㅂㅈㅅ/1.wav"
# output_path = "mixed_data/test1.wav"

# # 출력 폴더 생성
# os.makedirs(os.path.dirname(output_path), exist_ok=True)

# # 오디오 불러오기
# normal = AudioSegment.from_wav(normal_path)
# abnormal = AudioSegment.from_wav(abnormal_path)

# # 길이 맞추기
# min_len = min(len(normal), len(abnormal))
# normal = normal[:min_len]
# abnormal = abnormal[:min_len]

# # 오버레이 (동시에 재생)
# mixed = normal.overlay(abnormal)

# # 저장
# mixed.export(output_path, format="wav")
# print(f"Saved: {output_path}")

Saved: mixed_data/test1.wav


- 폴더 나뉘어져 있을 때 합치는 코드

In [ ]:
# # 폴더 나뉘었을 때 기준
# import os
# import random
# from glob import glob
# from pydub import AudioSegment

# # 입력/출력 디렉토리
# BASE_DIR = "data"
# OUTPUT_DIR = "mixed_data"
# os.makedirs(OUTPUT_DIR, exist_ok=True)

# # 조합 규칙 정의
# pairings = [
#     {
#         "normal": ("일상", "물 내리는 소리"),
#         "abnormal": ("강도",),
#         "label": "water_scream"
#     },
#     # 추가 매핑 필요 시 여기에 append
# ]

# def get_wav_files(base, *subdirs):
#     path = os.path.join(BASE_DIR, base, *subdirs)
#     return glob(os.path.join(path, "*.wav"))

# # 조합 및 저장
# for pair in pairings:
#     normal_files = get_wav_files("정상", *pair["normal"])
#     abnormal_files = get_wav_files("비정상", *pair["abnormal"])
    
#     # 짧은 쪽 기준으로 조합 수 제한
#     count = min(len(normal_files), len(abnormal_files))

#     for i in range(count):
#         normal = AudioSegment.from_wav(normal_files[i])
#         abnormal = AudioSegment.from_wav(abnormal_files[i])
        
#         # 길이 맞추기
#         min_len = min(len(normal), len(abnormal))
#         normal = normal[:min_len]
#         abnormal = abnormal[:min_len]

#         # overlay 후 저장
#         mixed = normal.overlay(abnormal)
#         output_path = os.path.join(OUTPUT_DIR, f"{pair['label']}_{i+1}.wav")
#         mixed.export(output_path, format="wav")

#         print(f"Saved: {output_path}")

### 소리 합성 (정상 + 정상 , 동시에 발생)

In [ ]:
import os
import random
from glob import glob
from pydub import AudioSegment

normal_dir = "종설데이터/정상"
output_dir = "종설데이터/혼합/정상"
os.makedirs(output_dir, exist_ok=True)

# 전체 정상 파일 리스트 확보
normal_files = glob(os.path.join(normal_dir, "**", "*.wav"), recursive=True)

# 안전한 샘플링
mix_count = min(len(normal_files), 30)
for i in range(mix_count):
    normal_path_1 = random.choice(normal_files)
    normal_path_2 = random.choice(normal_files)

    name_1 = os.path.splitext(os.path.basename(normal_path_1))[0]
    name_2 = os.path.splitext(os.path.basename(normal_path_2))[0]

    audio_1 = AudioSegment.from_wav(normal_path_1)
    audio_2 = AudioSegment.from_wav(normal_path_2)

    min_len = min(len(audio_1), len(audio_2))
    mixed = audio_1[:min_len].overlay(audio_2[:min_len])

    out_name = f"혼합_동시_정상_{name_1}_{name_2}_{i+1}.wav"
    out_path = os.path.join(output_dir, out_name)
    mixed.export(out_path, format="wav")
    print(f"Saved: {out_name}")


### 소리 합성 (비정상 + 비정상 , 동시에 발생)

In [ ]:
import os
import random
from glob import glob
from pydub import AudioSegment

abnormal_dir = "종설데이터/비정상"
output_dir = "종설데이터/혼합/비정상"
os.makedirs(output_dir, exist_ok=True)

# 전체 정상 파일 리스트 확보
abnormal_files = glob(os.path.join(abnormal_dir, "**", "*.wav"), recursive=True)

# 안전한 샘플링
mix_count = min(len(abnormal_files), 30)
for i in range(mix_count):
    abnormal_path_1 = random.choice(abnormal_files)
    abnormal_path_2 = random.choice(abnormal_files)

    name_1 = os.path.splitext(os.path.basename(abnormal_path_1))[0]
    name_2 = os.path.splitext(os.path.basename(abnormal_path_2))[0]

    audio_1 = AudioSegment.from_wav(abnormal_path_1)
    audio_2 = AudioSegment.from_wav(abnormal_path_2)

    min_len = min(len(audio_1), len(audio_2))
    mixed = audio_1[:min_len].overlay(audio_2[:min_len])

    out_name = f"혼합_동시_비정상_{name_1}_{name_2}_{i+1}.wav"
    out_path = os.path.join(output_dir, out_name)
    mixed.export(out_path, format="wav")
    print(f"Saved: {out_name}")

### 소리 합성 (비정상 + 비정상 , 연속 발생)

In [ ]:
import os
import random
from glob import glob
from pydub import AudioSegment

abnormal_dir = "종설데이터/비정상/비정상"
output_dir = "종설데이터/혼합"
os.makedirs(output_dir, exist_ok=True)

# 비정상 wav 파일 전체 불러오기
abnormal_files = glob(os.path.join(abnormal_dir, "**", "*.wav"), recursive=True)

# 샘플 수 설정
mix_count = min(len(abnormal_files), 30)
for i in range(mix_count):
    # 파일 두 개 랜덤 선택
    file_1 = random.choice(abnormal_files)
    file_2 = random.choice(abnormal_files)

    name_1 = os.path.splitext(os.path.basename(file_1))[0]
    name_2 = os.path.splitext(os.path.basename(file_2))[0]

    # 오디오 불러오기
    audio_1 = AudioSegment.from_wav(file_1)
    audio_2 = AudioSegment.from_wav(file_2)

    # 연속 붙이기
    combined = audio_1 + audio_2

    # 저장
    out_name = f"혼합_연속_비정상_{name_1}_then_{name_2}_{i+1}.wav"
    out_path = os.path.join(output_dir, out_name)
    combined.export(out_path, format="wav")
    print(f"Saved: {out_name}")
